In [2]:
cd D:\1MYFILES\ML_NGFW\FIREWALLS\-Phishing url detection

D:\1MYFILES\ML_NGFW\FIREWALLS\-Phishing url detection


In [3]:
!python --version 

Python 3.9.12


In [4]:
pip install seaborn 

Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd 
import numpy as np 
# import seaborn as sb # helps in statistical data visualization in different forms
import matplotlib.pyplot as plt # plots the data
%matplotlib inline 

import time 

from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import MultinomialNB # a simple learning algo which used bayes rule assuming the objects having some characteristics 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from nltk.tokenize import RegexpTokenizer # Tokenizes the url to get the required words
from nltk.stem.snowball import SnowballStemmer # stemms different works into meaningless words 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer # Vectorizes the tokens into a matrix 
from sklearn.pipeline import make_pipeline 

from bs4 import BeautifulSoup
!pip install selenium 
from selenium import webdriver
import networkx as nx # plots the internal link redirections within a website

import pickle # we'll store data in form of non-primitives and that has to be converted to character stream when we train it and "pickle" helps us to do that 
import warnings

In [14]:
phish_data = {}
phish_data = pd.read_csv('phishing_site_urls.csv')
phish_data

,URL,Label
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad
3,mail.printakid.com/www.online.americanexpress....,bad
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad
...,...,...
549341,23.227.196.215/,bad
549342,apple-checker.org/,bad
549343,apple-iclods.org/,bad
549344,apple-uptoday.org/,bad


In [15]:
# phish_data.info()

In [8]:
phish_data.isnull().sum()

URL      0
Label    0
dtype: int64

In [9]:
label_counts = pd.DataFrame(phish_data.Label.value_counts())
label_counts

,Label
good,392924
bad,156422


In [23]:
conda activate py3.5


Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install seaborn 

Note: you may need to restart the kernel to use updated packages.


In [26]:
import seaborn as sb 
sb.set_style('darkgrid')
sb.barplot(label_counts.index,label_counts.Label)
# sb.barplot( index , labels ) 

ImportError: cannot import name 'remove_na' from 'pandas.core.series' (C:\Users\lanka\anaconda3\lib\site-packages\pandas\core\series.py)

In [16]:
tokenizer = RegexpTokenizer(r'[A-Za-z]+') # tokenizes the given url based on the string argument

In [17]:
# tokenizing all the rows
# by passing all the URLs into the anonymous function i.e. mapping all the URLs into anonymous function.
# we use anonymous functions here for simplicity of code and no requirement of creating a function and storing it.
phish_data['text_tokenized'] = phish_data.URL.map(lambda t: tokenizer.tokenize(t)) # doing with all rows

In [18]:
phish_data.sample(5) 

,URL,Label,text_tokenized
13048,'taesx.com/img/?us.battle.net/login/en/?ref=ht...,bad,"[taesx, com, img, us, battle, net, login, en, ..."
66986,tools.ietf.org/html/rfc1082,good,"[tools, ietf, org, html, rfc]"
347222,google.com/finance?cid=16393552,good,"[google, com, finance, cid]"
293715,bodytronics.com/p/robic_stopwatches/RO501,good,"[bodytronics, com, p, robic, stopwatches, RO]"
387830,metacritic.com/redirectcritic?m=milk2008,good,"[metacritic, com, redirectcritic, m, milk]"


In [19]:
stemmer = SnowballStemmer("english")

In [20]:
# Stemming all the repetetive words into meaning less words
phish_data['text_stemmed'] = phish_data['text_tokenized'].map(lambda l: [stemmer.stem(word) for word in l])

In [21]:
phish_data.sample(5)

,URL,Label,text_tokenized,text_stemmed
351575,hiphopdx.com/index/news/id.12390/title.trackli...,good,"[hiphopdx, com, index, news, id, title, trackl...","[hiphopdx, com, index, news, id, titl, trackli..."
23194,freesteams.hut2.ru/,bad,"[freesteams, hut, ru]","[freesteam, hut, ru]"
438753,startribune.com/local/122631354.html,good,"[startribune, com, local, html]","[startribun, com, local, html]"
339225,flickr.com/photos/jonathan_cohen/4565948098/,good,"[flickr, com, photos, jonathan, cohen]","[flickr, com, photo, jonathan, cohen]"
194987,grenadamedia.com/,good,"[grenadamedia, com]","[grenadamedia, com]"


In [22]:
# joining all the list elements without any commas 
phish_data['text_sent'] = phish_data['text_stemmed'].map(lambda l: ' '.join(l))

In [24]:
phish_data.sample(20)

,URL,Label,text_tokenized,text_stemmed,text_sent
164209,delhi.quikr.com/kpmg-india-internship-program-...,good,"[delhi, quikr, com, kpmg, india, internship, p...","[delhi, quikr, com, kpmg, india, internship, p...",delhi quikr com kpmg india internship program ...
312733,davidson.k12.nc.us/education/school/school.php...,good,"[davidson, k, nc, us, education, school, schoo...","[davidson, k, nc, us, educ, school, school, ph...",davidson k nc us educ school school php sectionid
510936,unmannedsafetyinstitute.org/t76f3g,bad,"[unmannedsafetyinstitute, org, t, f, g]","[unmannedsafetyinstitut, org, t, f, g]",unmannedsafetyinstitut org t f g
379285,lonestarball.com/2011/7/7/2263647/series-previ...,good,"[lonestarball, com, series, preview, oakland, ...","[lonestarbal, com, seri, preview, oakland, ath...",lonestarbal com seri preview oakland athlet te...
212195,mediagallery.usatoday.com/2010-11-NHL-season/G...,good,"[mediagallery, usatoday, com, NHL, season, G]","[mediagalleri, usatoday, com, nhl, season, g]",mediagalleri usatoday com nhl season g
300590,cbssports.com/collegefootball/players/playerpa...,good,"[cbssports, com, collegefootball, players, pla...","[cbssport, com, collegefootbal, player, player...",cbssport com collegefootbal player playerpag s...
538847,aquatixbottle.com/nhftgrg45,bad,"[aquatixbottle, com, nhftgrg]","[aquatixbottl, com, nhftgrg]",aquatixbottl com nhftgrg
253662,upcoming.yahoo.com/event/8470024/CA/Oakland-CA...,good,"[upcoming, yahoo, com, event, CA, Oakland, CA,...","[upcom, yahoo, com, event, ca, oakland, ca, ch...",upcom yahoo com event ca oakland ca chris tuck...
421725,remembrance.vt.edu/biographies/couture-nowak.html,good,"[remembrance, vt, edu, biographies, couture, n...","[remembr, vt, edu, biographi, coutur, nowak, h...",remembr vt edu biographi coutur nowak html
270830,alacrastore.com/deal-snapshot/JD_Wetherspoon_P...,good,"[alacrastore, com, deal, snapshot, JD, Wethers...","[alacrastor, com, deal, snapshot, jd, wethersp...",alacrastor com deal snapshot jd wetherspoon pl...


In [ ]:
# DATA CLEANING
# noise = ["[]"]
# df = pd.read_csv('cleaned_data.csv', na_values = noise )
# df.isnull().sum()

In [ ]:
phish_data.info()

In [ ]:
ls -l

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
browser = webdriver.Chrome(r"chromedriver.exe")

In [ ]:
list_urls = ['https://ezee.com/cell-phones/','https://ezee.com/login.php?from=account.php%3Faction%3D'] #here i take phishing sites 
links_with_text = []

In [ ]:
for url in list_urls:
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,"html.parser")
    for line in soup.find_all('a' or 'link'):
        href = line.get('href')
        links_with_text.append([url, href])

In [ ]:
df = pd.DataFrame(links_with_text, columns=["from", "to"])
# DataFrame is 2D datastructure in pandas.   

In [ ]:
df.to_csv('dataframe.csv')

In [ ]:
# network graph using networkx 
GA = nx.from_pandas_edgelist(df, source="from", target="to")
nx.draw(GA, with_labels=False) # draws the internal hyperlink redirection network 

In [ ]:
cv = CountVectorizer() # converts the tokenized-stemmed words into a sparse matrix in a numerical format w.r.t frequency of word, as machine can't understand words

In [ ]:
feature = cv.fit_transform(phish_data.text_sent) # converts the tokens into algo-understandable numerical format by calculating different statistical terms like mean, standard deviation etc...,

In [11]:
feature[:5].toarray()

NameError: name 'feature' is not defined

In [ ]:
trainX, testX, trainY, testY = train_test_split(feature, phish_data.Label)
# trainX & testX are compressed sparse matrixes
# trainY & testY are splitted datasets 
# unable to store 1.4TiB data

In [ ]:
lr = LogisticRegression(max_iter = 549346 ) # Increasing the max_iter 

In [ ]:
# Training the lr model with training split data of X & Y 
lr.fit(trainX,trainY)
# ConvergenceWarning: lbfgs failed to converge (status=1): STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
# Increase the number of iterations (max_iter) or scale the data

In [ ]:
# from sklearn import preprocessing
# scaler = preprocessing.StandardScaler().fit(trainY) 
# scaled = scaler.transform(trainX) 

# cant enter strings to be scaled 
# sparse matrices can't be chosen 

In [ ]:
lr.score(trainX,trainY)

In [ ]:
lr.score(testX,testY) 
# accuracy improved from 0.960 --> 0.9664 on increasing the max_iter 

In [ ]:
# Storing the performance in dictionary 
score = {}
score['LogReg'] = np.round (lr.score(testX,testY) , 3 ) 

In [ ]:
score['LogReg']

In [ ]:
print('Training Accuracy :',lr.score(trainX,trainY))
print('Testing Accuracy :',lr.score(testX,testY))
con_mat = pd.DataFrame(confusion_matrix(lr.predict(testX), testY),
            columns = ['Predicted:Bad', 'Predicted:Good'],
            index = ['Actual:Bad', 'Actual:Good'])

print('\nCLASSIFICATION REPORT\n')
print(classification_report(lr.predict(testX), testY,
                            target_names =['Bad','Good']))

print('\nCONFUSION MATRIX')
plt.figure(figsize= (6,4))
sb.heatmap(con_mat, annot = True,fmt='d')

In [ ]:
# comparing the true labels i.e. true dataset with the predicted dataset and prepares the report of quality of prediction.
print(classification_report(testY, lr.predict(testX)))
# predictions will be done from sparse matrices. 
# Accuracy - out of all the good and bad predictions how many of them are acutally true
# Precision - out of all good/(+ve) predictions done how many of them are true +ve. similiarly for bad/(-ve)
# recall - it simply says how truthful the model is 
# recall - no. of predicted good / no.of actual real good  
# let's say no of +ve s are 6 but predicted are only 4 to be good 
# recall = 4 / 6 = 0.67
# f1 score = harmonic mean of precision & recall
# hm = 2*(h1 * h2 ) / ( h1 + h2 ) 


In [ ]:
## MULTINOMIAL NAIVE BAYES ALGORITHM 
mnb = MultinomialNB()
mnb.fit(trainX,trainY) 

In [ ]:
mnb.score(trainX,trainY)

In [ ]:
mnb.score(testX,testY)

In [33]:
score['multiNB'] = np.round(mnb.score(testX,testY), 2 )
score['multiNB']

NameError: name 'np' is not defined

In [ ]:
print('Training Accuracy :',mnb.score(trainX,trainY))
print('Testing Accuracy :',mnb.score(testX,testY))
con_mat = pd.DataFrame(confusion_matrix(mnb.predict(testX), testY),
            columns = ['Predicted:Bad', 'Predicted:Good'],
            index = ['Actual:Bad', 'Actual:Good'])

print('\nCLASSIFICATION REPORT\n')
print(classification_report(mnb.predict(testX), testY,
                            target_names =['Bad','Good']))

print('\nCONFUSION MATRIX')
plt.figure(figsize= (6,4))
sb.heatmap(con_mat, annot = True,fmt='d')

## LOGISTIC REGRESSION PIPELINE

In [ ]:
lr = LogisticRegression(max_iter = 549347)

In [ ]:
# pipeline = make_pipeline(cv(tokenizer.tokenize) , lr ) 
# arguments - ( preprocessing_techniques , model(i.e. estimator))
lr_pipeline = make_pipeline(CountVectorizer(tokenizer = RegexpTokenizer(r'[A-Za-z]+').tokenize,stop_words='english'), lr)
# lr_pipeline = make_pipeline(TfidfVectorizer(tokenizer = RegexpTokenizer(r'[A-Za-z]+').tokenize,stop_words='english'), lr)

In [ ]:
# INCLUDING SNOWBALLSTEMMER IN TRANSFORMERS 
lr_pipeline = make_pipeline(CountVectorizer(tokenizer = RegexpTokenizer(r'[A-Za-z]+').tokenize,SnowballStemmer(),stop_words='english'), lr)

In [ ]:
lr_pipeline = make_pipeline(TfidfTransformer(), lr)

In [ ]:
feature = cv.fit_transform(phish_data.text_sent) 

In [ ]:
# training the model with feature extractions and labelled data 
trainX, testX, trainY, testY = train_test_split(feature, phish_data.Label)

In [ ]:
lr_pipeline.fit(trainX,trainY)
lr_pipeline.score(testX,testY) 

In [ ]:
score['LogReg_tfid_feature_pipeline'] = lr_pipeline.score(testX,testY) 

In [ ]:
print('Training Accuracy :',lr_pipeline.score(trainX,trainY))
print('Testing Accuracy :',lr_pipeline.score(testX,testY))
con_mat = pd.DataFrame(confusion_matrix(lr_pipeline.predict(testX), testY),
            columns = ['Predicted:Bad', 'Predicted:Good'],
            index = ['Actual:Bad', 'Actual:Good'])


print('\nCLASSIFICATION REPORT\n')
print(classification_report(lr_pipeline.predict(testX), testY,
                            target_names =['Bad','Good']))

print('\nCONFUSION MATRIX')
plt.figure(figsize= (6,4))
sb.heatmap(con_mat, annot = True,fmt='d')

In [ ]:
pickle.dump(lr_pipeline, open('lr_model.pkl','wb'))

In [ ]:
lr_model = pickle.load(open('lr_model.pkl','rb'))
result = lr_model.score(testX, testY) 
print(result) 

In [ ]:
print(np.round(result , 2 ) )

In [ ]:
score['LogReg_pipeline'] = lr_model.score(testX, testY) 

In [ ]:
from sklearn import linear_model

reg = linear_model.BayesianRidge()
reg.fit(trainX,trainY)
reg.score(testX,testY)

## MULTINOMIAL-NB PIPELINE

In [ ]:
# COUNTVECTORIZER TRANSFORMER WITHOUT FEATURES EXTRACTED
mnb_pipeline = make_pipeline(CountVectorizer(tokenizer = RegexpTokenizer(r'[A-Za-z]+').tokenize,stop_words = 'english'),mnb)
trainX, testX, trainY, testY = train_test_split(phish_data.URL, phish_data.Label)

In [ ]:
mnb_pipeline.fit(trainX, trainY) 
mnb_pipeline.score(testX, testY)

In [ ]:
score['multiNB_pipeline'] = mnb_pipeline.score(testX, testY) 
pickle.dump(mnb_pipeline, open('mnb_model.pkl', 'wb'))

In [ ]:
# TFIDFTRANSFORMER WITH FEATURES EXTRACTED
mnb_pipeline = make_pipeline(TfidfTransformer(),mnb)
trainX, testX, trainY, testY = train_test_split(feature, phish_data.Label)

In [ ]:
mnb_pipeline.fit(trainX, trainY) 
mnb_pipeline.score(testX, testY)

In [ ]:
score['multiNB_tfid_feature_pipeline'] = mnb_pipeline.score(testX, testY) 
pickle.dump(mnb_pipeline, open('mnb_model.pkl', 'wb'))

In [ ]:
# TFIDFVECTORIZER TRANSFORMER WITH FEATURES EXTRACTED
# mnb_pipeline = make_pipeline(TfidfVectorizer(tokenizer = RegexpTokenizer(r'[A-Za-z]+').tokenize,stop_words = 'english'),mnb)
# trainX, testX, trainY, testY = train_test_split(feature, phish_data.Label)
# VECTORIZERS NEED LOWER BOUND WITH THE FEATURE EXTRACTION DATA

In [7]:
import pickle
mnb_model = pickle.load(open('mnb_model.pkl', 'rb'))
# mnb_model.score(testX,testY) 

In [ ]:
print('Training Accuracy :',mnb_pipeline.score(trainX,trainY))
print('Testing Accuracy :',mnb_pipeline.score(testX,testY))
con_mat = pd.DataFrame(confusion_matrix(mnb_pipeline.predict(testX), testY),
            columns = ['Predicted:Bad', 'Predicted:Good'],
            index = ['Actual:Bad', 'Actual:Good'])

print('\nCLASSIFICATION REPORT\n')
print(classification_report(mnb_pipeline.predict(testX), testY,
                            target_names =['Bad','Good']))

print('\nCONFUSION MATRIX')
plt.figure(figsize= (6,4))
sb.heatmap(con_mat, annot = True,fmt='d')

In [9]:
payload = ['yeniik.com.tr/wp-admin/js/login.alibaba.com/login.jsp.php']
print( mnb_model.predict(payload) )

['bad']


## BERT TRANSFORMER

In [5]:
cd D:\1MYFILES\ML_NGFW\FIREWALLS\-Phishing url detection

D:\1MYFILES\ML_NGFW\FIREWALLS\-Phishing url detection


In [6]:
try:
    import numpy as np
    import pandas as pd

    import torch
    import transformers as ppb # pytorch transformers
    
    
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import cross_val_score
    
    from sklearn.preprocessing import LabelEncoder
    from sklearn.model_selection import train_test_split
    
    from sklearn.naive_bayes import MultinomialNB
    
    import warnings

    import swifter
    import tqdm
    tqdm.pandas()

    warnings.filterwarnings('ignore')
except Exception  as e: pass

In [7]:
df = {}
df = pd.read_csv('phishing_site_urls.csv')
# df = df.dropna(how='all')

In [9]:
X = df[1]
Y = df[2]
encoder = LabelEncoder()
Y = encoder.fit_transform(Y)

KeyError: 1

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [11]:
class BertTokenizer(object):

    def __init__(self, text=[]):
        self.text = text

        # For DistilBERT:
        self.model_class, self.tokenizer_class, self.pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

        # Load pretrained model/tokenizer
        self.tokenizer = self.tokenizer_class.from_pretrained(self.pretrained_weights)

        self.model = self.model_class.from_pretrained(self.pretrained_weights)

    def get(self):

        df = pd.DataFrame(data={"text":self.text})
        tokenized = df["text"].swifter.apply((lambda x: self.tokenizer.encode(x, add_special_tokens=True)))

        max_len = 0
        for i in tokenized.values:
            if len(i) > max_len:
                max_len = len(i)

        padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

        attention_mask = np.where(padded != 0, 1, 0)
        input_ids = torch.tensor(padded)
        attention_mask = torch.tensor(attention_mask)

        with torch.no_grad(): last_hidden_states = self.model(input_ids, attention_mask=attention_mask)
        
        features = last_hidden_states[0][:, 0, :].numpy()

        return features

In [12]:
_instance =BertTokenizer(text=x_train)
tokens = _instance.get()

NameError: name 'x_train' is not defined

# MODEL

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(tokens, y_train)

# TEST

In [ ]:
_instance =BertTokenizer(text=x_test)
tokensTest = _instance.get()

In [ ]:
predicted = lr_clf.predict(tokensTest)

In [ ]:
np.mean(predicted == y_test)

# Performance evaluation of all models

In [ ]:
acc = pd.DataFrame.from_dict(score,orient = 'index',columns=['Accuracy'])
sb.set_style('darkgrid')
sb.barplot(acc.index,acc.Accuracy)
print(acc)

In [ ]:
%pip install pyngrok nest_asyncio fastapi uvicorn loguru
import uvicorn
from fastapi import FastAPI
import joblib,os
from fastapi import FastAPI
from pydantic import BaseModel
from loguru import logger

app = FastAPI()

#pkl
phish_model = open('lr_model.pkl','rb')
phish_model_ls = joblib.load(phish_model)

# ML Aspect
@app.get('/predict/{feature}')
async def predict(features):
	X_predict = []
	X_predict.append(str(features))
	y_Predict = phish_model_ls.predict(X_predict)
	if y_Predict == 'bad':
		result = "This is a Phishing Site"
	else:
		result = "This is not a Phishing Site"

	return (features, result)
if __name__ == '__main__':
	uvicorn.run(app,host="127.0.0.1",port=8000)